In [1]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)   # Tensorflow의 버전 출력

mnist = keras.datasets.mnist

# MNIST 데이터를 로드. 다운로드하지 않았다면 다운로드까지 자동으로 진행됩니다. 
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

print("x_train 배열의 크기:", len(x_train))  # x_train 배열의 크기 출력

2.6.0
x_train 배열의 크기: 60000


In [2]:
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))

최소값: 0.0  최대값: 1.0


In [20]:
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

In [58]:
# 바꿔 볼 수 있는 하이퍼파라미터들 - 숫자를 다양하게 바꿔 보세요. 
n_channel_1 = 160
n_channel_2 = 32
n_dense = 480
n_train_epoch= 10

In [59]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 확인
print('Layer: ', len(model.layers))
print(model.summary())

Layer:  7
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 160)       1600      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 160)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        46112     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 480)               384480    
_________________________________________________________________
dense_5 (Dense)              (None, 10)     

In [60]:
# 모델 컴파일
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_reshaped, y_train, epochs=n_train_epoch)

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose=2)

print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1141 - accuracy: 0.9648
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0380 - accuracy: 0.9880
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0265 - accuracy: 0.9917
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0180 - accuracy: 0.9942
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0155 - accuracy: 0.9951
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0111 - accuracy: 0.9965
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0093 - accuracy: 0.9970
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0081 - accuracy: 0.9974
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0081 - accuracy: 0.9974
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.005

192, 96, 320, 10:  
- test_loss: 0.036911625415086746  
- test_accuracy: 0.9916999936103821  

224, 192, 512, 10:
- test_loss: 0.04630236327648163 
- test_accuracy: 0.9911999702453613

240, 256, 288, 10:

- test_loss: 0.034723758697509766
- test_accuracy: 0.9912999868392944

In [ ]:
160, 32, 480, 10:



In [4]:
from tensorflow.keras import layers

In [47]:
def build_model(hp):

    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(
        hp.Int("n_ch1", min_value=16, max_value=256, step=16),
        (3,3), activation='relu', input_shape=(28,28,1)))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(
        hp.Int("n_ch2", min_value=32, max_value=512, step=32), 
        (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
        hp.Int("n_dense", min_value=32, max_value=512, step=32),
        activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model


In [11]:
!pip install keras-tuner -q

In [48]:
import keras_tuner

build_model(keras_tuner.HyperParameters())

In [49]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir_e10",
    project_name="tune_hyperparam",
)

In [50]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
n_ch1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
n_ch2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
n_dense (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [51]:
tuner.search(x_train_reshaped, y_train, epochs=n_train_epoch, validation_data=(x_test_reshaped, y_test))

Trial 3 Complete [00h 02m 13s]
val_accuracy: 0.9925499856472015

Best val_accuracy So Far: 0.9928499758243561
Total elapsed time: 00h 09m 07s
INFO:tensorflow:Oracle triggered exit


In [29]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]

best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 192)       1920      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 192)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 96)        165984    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 96)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 320)               768320    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [30]:
second_model = models[1]

second_model.build(input_shape=(None, 28, 28))
second_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 224)       2240      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 224)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 192)       387264    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 192)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4800)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2458112   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

In [53]:
models_e10 = tuner.get_best_models(num_models=2)
best_model_e10 = models_e10[0]

best_model_e10.build(input_shape=(None, 28, 28))
best_model_e10.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 240)       2400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 240)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 256)       553216    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 288)               1843488   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [54]:
second_model_e10 = models_e10[1]

second_model_e10.build(input_shape=(None, 28, 28))
second_model_e10.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 160)       1600      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 160)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        46112     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 480)               384480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                4